### 🧠 What is Query Decomposition?
Query decomposition is the process of taking a complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

#### ✅ Why Use Query Decomposition?

- Complex queries often involve multiple concepts

- LLMs or retrievers may miss parts of the original question

- It enables multi-hop reasoning (answering in steps)

- Allows parallelism (especially in multi-agent frameworks)

In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

/Users/huzi/Desktop/RAG/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Step 1: Load and embed the document
loader = TextLoader("langchain_crewai_dataset.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.7})

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model(model="groq:meta-llama/Llama-4-Scout-17B-16E-Instruct")
llm

ChatGroq(profile={}, client=<groq.resources.chat.completions.Completions object at 0x12b558e80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x12b558910>, model_name='meta-llama/Llama-4-Scout-17B-16E-Instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
# Step 3: Query decomposition
decomposition_prompt = PromptTemplate.from_template("""
You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

Question: "{question}"

Sub-questions:
""")
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [12]:
query = "How does LangChain use memory and agents compared to CrewAI?"
decomposition_question=decomposition_chain.invoke({"question": query})


In [13]:
print(decomposition_question)

To tackle the complex question of how LangChain uses memory and agents compared to CrewAI, we can break it down into more manageable sub-questions. Here are 3 sub-questions for better document retrieval:

1. **How does LangChain utilize memory in its operations?**
   - This sub-question aims to understand the specific mechanisms, techniques, or architectures LangChain employs for memory usage.

2. **How does LangChain implement and use agents in its framework?**
   - This sub-question seeks to clarify the role of agents within LangChain, including their design, functionality, and integration with other components.

3. **How do CrewAI's approaches to memory and agents differ from or compare to LangChain's?**
   - Alternatively, this could be split into two sub-questions for a more granular comparison:
   - **How does CrewAI utilize memory in its operations?**
   - **How does CrewAI implement and use agents in its framework?**

By answering these sub-questions, one can gather detailed in

In [14]:
# Step 4: QA chain per sub-question
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [15]:
# Step 5: Full RAG pipeline logic
def full_query_decomposition_rag_pipeline(user_query):
    # Decompose the query
    sub_qs_text = decomposition_chain.invoke({"question": user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]
    
    results = []
    for subq in sub_questions:
        docs = retriever.invoke(subq)
        result = qa_chain.invoke({"input": subq, "context": docs})
        results.append(f"Q: {subq}\nA: {result}")
    
    return "\n\n".join(results)

In [16]:
# Step 6: Run
query = "How does LangChain use memory and agents compared to CrewAI?"
final_answer = full_query_decomposition_rag_pipeline(query)
print("✅ Final Answer:\n")
print(final_answer)

✅ Final Answer:

Q: To tackle the complex question of how LangChain uses memory and agents compared to CrewAI, we can break it down into more manageable sub-questions. Here are 3 sub-questions that can help in better document retrieval:
A: Based on the provided context, here are 3 sub-questions that can help in better understanding how LangChain uses memory and agents compared to CrewAI:

1. **How does LangChain integrate with vector databases, and what benefits does this provide for semantic search and Retrieval-Augmented Generation (RAG)?**

This sub-question can help clarify LangChain's approach to memory and knowledge retrieval, specifically its use of vector databases like FAISS, Chroma, Pinecone, and Weaviate.

2. **What role does LangChain play in handling retrieval and tool wrapping in hybrid systems with CrewAI, and how does CrewAI manage role-based collaboration?**

This sub-question can help understand the division of labor between LangChain and CrewAI in hybrid systems, and